In [1]:
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# Inisialisasi
lemmatizer = WordNetLemmatizer()
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("intents.json").read()
intents = json.loads(data_file)

# 2. Data pre-processing

In [2]:
# Proses setiap intent dan pattern
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        # Tambahkan ke dokumen dan kumpulkan kata-kata
        words.extend(w)
        documents.append((w, intent['tag']))
        # Tambahkan kelas unik
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

### Tokenisasi

- Pada proses tokenisasi pada dasarnya adalah pemisahan kalimat, paragraf, 
atau seluruh dokumen teks menjadi unit yang lebih kecil, proses itu yang disebut token

- Pada proses ini juga akan save documen tersebut menjadi file label.pkl dan texts.pkl (proses labeling)

In [3]:
# Buat daftar kata yang unik dan urutkan
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(f"{len(documents)} documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

# Simpan kata dan kelas ke file dengan pickle
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

142 documents
33 classes ['Batuk', 'CPR', 'Cedera Kepala', 'Cedera otot', 'Demam', 'Diare', 'Dingin', 'Gigitan Serangga', 'Hidung Tersumbat', 'Keseleo', 'Lecet', 'Luka', 'Luka Bakar Kimia', 'Masalah Pencernaan', 'Masalah kulit', 'Mata', 'Memar', 'Nyeri Perut', 'Nyeri Testis', 'Pendarahan Normal', 'Racun', 'Ruam', 'Sakit Tenggorokan', 'Sakit kepala', 'Sengatan', 'Serpihan', 'Tenggelam', 'Terbakar Matahari', 'Tersedak', 'gigitan binatang', 'gigitan ular', 'kejang', 'pingsan']
111 unique lemmatized words ['air', 'akibat', 'aku', 'alergi', 'ambil', 'anda', 'anjing', 'antara', 'apa', 'atau', 'bagaimana', 'bahan', 'baik', 'bakar', 'batuk', 'berapa', 'berlangsung', 'binatang', 'cara', 'cedera', 'cpr', 'dalam', 'demam', 'dengan', 'di', 'diare', 'digigit', 'digunakan', 'dilakukan', 'diminum', 'dioleskan', 'diterapkan', 'e', 'flu', 'gigitan', 'harus', 'hewan', 'hidung', 'ingin', 'jika', 'karena', 'kejang', 'kepala', 'keracunan', 'keseleo', 'kimia', 'krim', 'kulit', 'lakukan', 'lama', 'langkah', 

# 3. Creating Training Data

- Pada dasarnya, bag of words adalah representasi sederhana dari setiap teks dalam sebuah kalimat sebagai bag of words-nya.

In [4]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # Buat 'bag' dengan panjang tetap
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # Pastikan panjang 'bag' dan 'output_row' seragam
    if len(bag) == len(words):  # Verifikasi panjang bag
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        
        if len(output_row) == len(classes):  # Verifikasi panjang output_row
            training.append([bag, output_row])

random.shuffle(training)

# Ubah training menjadi numpy array hanya setelah memastikan bahwa setiap elemen seragam
training = np.array(training, dtype=object)

# Pisahkan train_x dan train_y
train_x = np.array([item[0] for item in training], dtype=np.float32)
train_y = np.array([item[1] for item in training], dtype=np.float32)

print(train_x.shape)  # Verifikasi bentuk train_x
print(train_y.shape)  # Verifikasi bentuk train_y


(142, 111)
(142, 33)


# 5. Creating Modeling 

- Pada proses ini kami akan membuat model jaringan saraf dan menyimpan model tersebut 

In [8]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('models.keras', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0260 - loss: 3.5452       
Epoch 2/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0434 - loss: 3.4550
Epoch 3/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1170 - loss: 3.4141
Epoch 4/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - accuracy: 0.1100 - loss: 3.3531
Epoch 5/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step - accuracy: 0.1143 - loss: 3.3243
Epoch 6/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1358 - loss: 3.2270 
Epoch 7/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2123 - loss: 3.0957
Epoch 8/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2071 - loss: 3.0043   
Epoch 9/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2143 - loss: 2.8626   
Epoch 10/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 932us/step - accuracy: 0.2105 - loss: 2.8489
Epoch 11/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2563 - loss: 2.7579   
Epoch 12/200
29/29 ━━━━━━━━━━━━━━━━━━━━